In [1]:
#SA Bond pricing formula

#https://clientportal.jse.co.za/Content/JSEValuations%20Methodologies/Bond%20Pricing%20Formula%20-%20Specifications.pdf




In [2]:
#Inputs

Yeild_to_maturity = 7.5
Settlement_date = '26 August 2005'
Bond = 'R186'
Maturity_date = '21 December 2026'
Coupon = 10.5
Coupon_dates = ['21 June','21 Dec']
BcDates = ['11 June','11 December']
Redemption_amount = 100
PROUND = 5
Nominal = 1.5e6



In [3]:
#Calculations

Coupon_dates[1]

'21 Dec'

In [4]:
#Date manipulation

from dateutil import parser
import datetime

dt = parser.parse(Settlement_date,)
dt.year
M_b = parser.parse(Maturity_date,)


def convert_date(date_string):
    import QuantLib as ql
    from dateutil import parser
    
    temp_date = parser.parse(date_string + " " + str(dt.year),)
    temp_date = ql.Date(temp_date.day,temp_date.month,temp_date.year)
    return temp_date



In [5]:
#Calendars
import QuantLib as ql

date = ql.Date(dt.day,dt.month,dt.year)
M_b = ql.Date(M_b.day,M_b.month,M_b.year)
calendar = ql.SouthAfrica()
is_business_day = calendar.isBusinessDay(date)
print(f"Is {date} a business day? {'Yes' if is_business_day else 'No'}")

Is August 26th, 2005 a business day? Yes


In [28]:
M_b.month()

12

In [6]:
#Coupon dates
Coupon_date_1 = parser.parse(Coupon_dates[0] + " " + str(dt.year),)
Coupon_date_1 = ql.Date(Coupon_date_1.day,Coupon_date_1.month,Coupon_date_1.year)

Coupon_date_2 = parser.parse(Coupon_dates[1] + " " + str(dt.year),)
Coupon_date_2 = ql.Date(Coupon_date_2.day,Coupon_date_2.month,Coupon_date_2.year)

def LCD(Date,Coupon_date_1,Coupon_date_2):
    max_coupon_date = max(Coupon_date_1,Coupon_date_2)
    min_coupon_date = min(Coupon_date_1,Coupon_date_2)
    if (date >= min_coupon_date):
            return min_coupon_date   
    else :
         return max_coupon_date 

# def NCD(Date,Coupon_date_1,Coupon_date_2):
#     return(max(Date,Coupon_date_1,Coupon_date_2))


last_cd = LCD(Date = date,Coupon_date_1=Coupon_date_1,Coupon_date_2=Coupon_date_2)
# next_cd = NCD(Date = date,Coupon_date_1=Coupon_date_1,Coupon_date_2=Coupon_date_2)
next_cd = calendar.advance(last_cd,ql.Period(6,ql.Months))
print(last_cd,next_cd)

June 21st, 2005 December 21st, 2005


In [7]:
#Books closed dates
BCD_1 = convert_date(BcDates[0])
BCD_2 = convert_date(BcDates[1])



In [8]:
#Number of remaining coupons

N = round((M_b-next_cd)/(365.25/2),0)
print(N)

42.0


In [9]:
#Proximity
prox_cpn_dt1 = date - Coupon_date_1
prox_cpn_dt2 = date - Coupon_date_2

bcd_use = next_cd -10
print(bcd_use)

December 11th, 2005


In [10]:
#Cumex

if (date < bcd_use):
    cumex = 1
else:
    cumex = 0



In [11]:
#DaysAccrued

if (cumex ==1):
    daysacc = (date - last_cd)
else:
    daysacc = (date - next_cd)


In [12]:
#Coupon
CPN = Coupon/2


In [13]:
#Coupon payable on NCD
cpn_at_ncd = CPN*cumex

In [14]:
#Semi +- annual discount factor
F = 1/(1+Yeild_to_maturity/200)

In [15]:
#Broken period

if (next_cd != M_b):
    BP = (next_cd-date)/(next_cd - last_cd)
else:
    BP = (next_cd-date)/(365/2)
    

In [16]:
#Broken period discount factor
if (next_cd != M_b):
    BPF = F**BP
else:
    BPF = F/(F+BP*(1-F))
    

In [17]:
#Differentials
#The first differential of BPF with respect to F, dBPF = ∂BPF/∂F

if (next_cd != M_b):
    dBPF = (BP*BPF)/F
else:
    dBPF = (BP*(BPF**2))/(F**2)

In [18]:
#Differentials
#The second differential of BPF with respect to F, d2BPF = ∂2BPF/∂F2

if (next_cd != M_b):
    d2BPF = dBPF*(BP-1)/F
else:
    d2BPF = dBPF*(BP*BPF-F)/(F**2)

if (F!=1):
    dCPN = CPN*(1-(N-N*F +1)*F**N)/((1-F)**2)
else:
    dCPN = CPN*N(N**2-1)/3

dR = N*Redemption_amount*F**(N-1)

d2R = N*(N-1)*Redemption_amount*(F**(N-2))

if (F!=1):
    d2CPN = CPN*(2-(N*(1-F)*(2+(N-1)*(1-F))+2*F)*(F**(N-1)))/(1-F)**3
else:
    d2CPN = CPN*N*(N**2-1)/3


In [19]:
#Results

#Unrounded and rounded accrued interest

accrint = (daysacc*Coupon)/365
raccrint = round(accrint,PROUND)

In [20]:
#Unrounded all-in price
if (F!=1):
    AIP = BPF*(cpn_at_ncd+CPN*F*(1-F**N)/(1-F)+Redemption_amount*(F**N))
else:
    AIP = cpn_at_ncd + Coupon*N + Redemption_amount

In [21]:
#Unrounded clean price

CP = AIP - accrint

In [22]:
#Rounded clean price
Rounded_CP = round(CP,PROUND)

In [23]:
#Rounded All-in-price

Rounded_AIP = Rounded_CP + raccrint

In [24]:
#Consideration

#Interest consideration

IntConsid = round(raccrint*Nominal/100,2)

#All-in consideration

All_in_Consid = round(Rounded_AIP*Nominal/100,2)

#Clean consideration

CleanConsid = All_in_Consid - IntConsid

In [25]:
#Sensitivities

dAIP = dBPF*AIP/BPF + BPF*(dCPN+dR)

d2AIP = d2BPF*AIP/BPF+ dBPF*((BPF*dAIP-AIP*dBPF)/BPF**2 + dCPN + dR)+BPF*(d2CPN+d2R)

Delta = -1*(F**2)*dAIP/200

In [26]:
#Duration and complexity
Dmod = -100*Delta/AIP
Dur = Dmod/F
Rand_per_point = Delta*100

d2AIPy = (dAIP*(F**3)/2 + d2AIP*(F**4)/4)/10000

Conv = (10000/AIP)*d2AIPy